# 📊 Analisis SI Channel Potensial — INAPROC 2025 (Realisasi)
### Telkomsel Enterprise | Bid Management — Data Science

**Objective:** Menarik calon SI (System Integrator) potensial dari data realisasi INAPROC 2025
untuk KLPD yang potensial di tahun 2026.

**Dua Perspektif Analisis:**
1. **Pemerintah Daerah** — per Zona Wilayah (Sumatera, Kalimantan, Jawa, Bali-NusRa, Sulawesi, Papua-Maluku)
2. **Pemerintah Pusat** — per Bidang Kenaikan Tertinggi (Pendidikan, Pertahanan, Kesehatan, Energi, Infrastruktur)

**Rules:**
- Nama_Pemenang **wajib terisi** (bukan kosong)
- Pemenang harus melayani **≥5 Satuan Kerja** berbeda
- Klasifikasi **ICT vs Non-ICT** via whitelist + blacklist
- Filter Instansi_Pembeli & Satuan_Kerja berdasarkan **keyword dinas/K-L relevan** (tanpa singkatan)

In [1]:
import pandas as pd
import sqlite3
import re
import os
from datetime import datetime

print("=" * 70)
print("  ANALISIS SI CHANNEL POTENSIAL — INAPROC 2025")
print("  Telkomsel Enterprise | Bid Management — Data Science")
print("=" * 70)

  ANALISIS SI CHANNEL POTENSIAL — INAPROC 2025
  Telkomsel Enterprise | Bid Management — Data Science


---
## 1. KONFIGURASI — Zona Wilayah & Bidang Fokus

### 1A. Pemerintah Daerah — Zona Wilayah
Keyword dinas relevan per zona. Digunakan untuk filter kolom `Satuan_Kerja` dan `Instansi_Pembeli`.

**Cara kerja filter:**
- Kolom `Lokasi` → extract Provinsi → mapping ke Zona
- Kolom `Instansi_Pembeli` berisi: `Provinsi xxx`, `Kab. xxx`, `Kota xxx` → mapping ke Zona  
- Kolom `Satuan_Kerja` di-match dengan keyword dinas relevan
- Satker yang **TIDAK mengandung** keyword relevan → **dihapus**

In [2]:
# =============================================================================
# ZONA WILAYAH — KEYWORD DINAS LENGKAP (TANPA SINGKATAN)
# =============================================================================

ZONA_WILAYAH = {
    "Sumatera": {
        "provinsi": [
            "Aceh", "Sumatera Utara", "Sumatera Barat", "Riau",
            "Kepulauan Riau", "Jambi", "Sumatera Selatan",
            "Bangka Belitung", "Bengkulu", "Lampung",
        ],
        "fokus": "Revitalisasi sekolah, irigasi, dan koperasi lokal",
        "dinas_keywords": [
            # ── Pendidikan ──
            # Clue: Dinas Pendidikan, Dinas Pendidikan dan Kebudayaan, 
            #        DIKBUD, sekolah, PAUD, TK, SD, SMP, SMA
            "PENDIDIKAN", "SEKOLAH", "KEBUDAYAAN",
            "PAUD", "TAMAN KANAK", "DIKBUD", "DIKNAS",
            "DIKDASMEN",

            # ── Pekerjaan Umum dan Penataan Ruang ──
            # Clue: Dinas Pekerjaan Umum, Dinas PUPR, Dinas PU,
            #        Bina Marga, Cipta Karya, Sumber Daya Air, Penataan Ruang
            "PEKERJAAN UMUM", "PENATAAN RUANG", "BINA MARGA",
            "CIPTA KARYA", "SUMBER DAYA AIR", "PERUMAHAN RAKYAT",
            "KAWASAN PERMUKIMAN", "KAWASAN PEMUKIMAN",

            # ── Pertanian, Perkebunan, Peternakan ──
            # Clue: Dinas Pertanian, Dinas Tanaman Pangan, Dinas Peternakan,
            #        Dinas Perkebunan, Ketahanan Pangan, Hortikultura
            "PERTANIAN", "TANAMAN PANGAN", "HORTIKULTURA",
            "PETERNAKAN", "PERKEBUNAN", "KETAHANAN PANGAN",
            "PENYULUHAN",

            # ── Koperasi ──
            # Clue: Dinas Koperasi, UMKM, Usaha Mikro, Usaha Kecil
            "KOPERASI", "USAHA MIKRO", "USAHA KECIL", "UMKM",

            # ── Irigasi & Pengairan ──
            # Clue: Dinas Pengairan, irigasi, bendungan, waduk, sungai
            "IRIGASI", "PENGAIRAN", "BENDUNGAN", "WADUK", "SUNGAI",
        ],
        "produk_tsel": "IoT Smart Farming, Fleet Management, Telkomsel Learning Platform, IoT Smart Water Meter",
    },

    "Kalimantan": {
        "provinsi": [
            "Kalimantan Barat", "Kalimantan Tengah", "Kalimantan Selatan",
            "Kalimantan Timur", "Kalimantan Utara",
        ],
        "fokus": "Infrastruktur dasar, energi, dan transportasi pendukung IKN",
        "dinas_keywords": [
            # ── Pekerjaan Umum dan Penataan Ruang ──
            # Clue: sama seperti Sumatera + fokus infrastruktur jalan, jembatan
            "PEKERJAAN UMUM", "PENATAAN RUANG", "BINA MARGA",
            "CIPTA KARYA", "SUMBER DAYA AIR", "PERUMAHAN RAKYAT",
            "KAWASAN PERMUKIMAN", "KAWASAN PEMUKIMAN",
            "JALAN", "JEMBATAN",

            # ── Perhubungan & Transportasi ──
            # Clue: Dinas Perhubungan, terminal, pelabuhan, bandara, 
            #        transportasi darat, transportasi laut, navigasi
            "PERHUBUNGAN", "TRANSPORTASI", "PELABUHAN",
            "BANDARA", "TERMINAL", "NAVIGASI",
            "LALU LINTAS", "ANGKUTAN",

            # ── Energi dan Sumber Daya Mineral ──
            # Clue: Dinas ESDM, Dinas Energi, pertambangan, 
            #        kelistrikan, listrik pedesaan, mineral, migas
            "ENERGI", "SUMBER DAYA MINERAL", "PERTAMBANGAN",
            "KELISTRIKAN", "LISTRIK", "MIGAS", "MINERAL",
            "GEOLOGI",

            # ── Otorita IKN ──
            # Clue: Otorita Ibu Kota Nusantara, IKN, Nusantara
            "OTORITA", "IBU KOTA NUSANTARA", "NUSANTARA",
        ],
        "produk_tsel": "IoT Smart City, Industrial IoT, IoT Smart Energy Meter",
    },

    "Jawa": {
        "provinsi": [
            "DKI Jakarta", "Jawa Barat", "Jawa Tengah",
            "DI Yogyakarta", "Jawa Timur", "Banten",
        ],
        "fokus": "Megalopolis nasional — pusat industri teknologi dan ekonomi kreatif",
        "dinas_keywords": [
            # ── Komunikasi dan Informatika ──
            # Clue: Diskominfo, Dinas Komunikasi, Dinas Informatika,
            #        statistik, persandian, e-government
            "KOMUNIKASI", "INFORMATIKA", "KOMINFO", "DISKOMINFO",
            "STATISTIK", "PERSANDIAN", "E-GOVERNMENT",

            # ── Perindustrian dan Perdagangan ──
            # Clue: Dinas Perindustrian, Dinas Perdagangan, industri,
            #        ekonomi kreatif, pasar, UMKM
            "PERINDUSTRIAN", "PERDAGANGAN", "INDUSTRI",
            "EKONOMI KREATIF", "PASAR",

            # ── Pendapatan Daerah ──
            # Clue: Bappenda, Badan Pendapatan Daerah, pajak daerah,
            #        retribusi, keuangan daerah, BPKAD
            "PENDAPATAN DAERAH", "BAPPENDA", "PAJAK",
            "RETRIBUSI", "KEUANGAN DAERAH", "BPKAD",

            # ── Perencanaan & Riset ──
            # Clue: Bappeda, Badan Perencanaan Pembangunan, 
            #        riset, inovasi, litbang
            "PERENCANAAN PEMBANGUNAN", "BAPPEDA", "RISET",
            "INOVASI", "LITBANG", "PENELITIAN",
        ],
        "produk_tsel": "Omnichannel, Msight/Tsurvey, IoT Monitoring Management",
    },

    "Bali — NusRa": {
        "provinsi": [
            "Bali", "Nusa Tenggara Barat", "Nusa Tenggara Timur",
        ],
        "fokus": "Peningkatan kualitas pendidikan, gizi, dan koperasi pariwisata lokal",
        "dinas_keywords": [
            # ── Pendidikan ──
            "PENDIDIKAN", "SEKOLAH", "KEBUDAYAAN",
            "PAUD", "DIKBUD", "DIKNAS",

            # ── Kesehatan & Gizi ──
            # Clue: Dinas Kesehatan, rumah sakit, RSUD, puskesmas,
            #        gizi, stunting, KB, keluarga berencana
            "KESEHATAN", "RUMAH SAKIT", "RSUD", "PUSKESMAS",
            "GIZI", "STUNTING", "KELUARGA BERENCANA",

            # ── Pariwisata & Koperasi ──
            # Clue: Dinas Pariwisata, wisata, koperasi, UMKM, 
            #        ekonomi kreatif, kebudayaan, adat
            "PARIWISATA", "WISATA", "KOPERASI",
            "EKONOMI KREATIF", "ADAT", "UMKM",

            # ── Sosial ──
            # Clue: Dinas Sosial, pemberdayaan masyarakat, PKH
            "SOSIAL", "PEMBERDAYAAN MASYARAKAT", "PEMBERDAYAAN PEREMPUAN",
        ],
        "produk_tsel": "DigiAds, Msight/Tsurvey, IoT Smart Connectivity, Omnichannel",
    },

    "Sulawesi": {
        "provinsi": [
            "Sulawesi Utara", "Sulawesi Tengah", "Sulawesi Selatan",
            "Sulawesi Tenggara", "Gorontalo", "Sulawesi Barat",
        ],
        "fokus": "Sekolah rakyat, irigasi pertanian, dan smart tourism infrastructure",
        "dinas_keywords": [
            # ── Pendidikan ──
            "PENDIDIKAN", "SEKOLAH", "KEBUDAYAAN",
            "PAUD", "DIKBUD", "DIKNAS",

            # ── Pekerjaan Umum dan Penataan Ruang ──
            "PEKERJAAN UMUM", "PENATAAN RUANG", "BINA MARGA",
            "CIPTA KARYA", "SUMBER DAYA AIR",
            "KAWASAN PERMUKIMAN", "KAWASAN PEMUKIMAN",

            # ── Pariwisata ──
            "PARIWISATA", "WISATA", "TOURISM",

            # ── Pertanian, Kelautan, Irigasi ──
            "PERTANIAN", "KELAUTAN", "PERIKANAN",
            "IRIGASI", "PENGAIRAN", "PERKEBUNAN",
            "KETAHANAN PANGAN", "TANAMAN PANGAN",
        ],
        "produk_tsel": "IoT FleetSight, IoT Smart Connectivity, IoT Smart Water Meter, Msight/TSurvey",
    },

    "Papua — Maluku": {
        "provinsi": [
            "Papua", "Papua Barat", "Papua Selatan", "Papua Tengah",
            "Papua Pegunungan", "Papua Barat Daya",
            "Maluku", "Maluku Utara",
        ],
        "fokus": "Pendidikan & kesehatan dasar, perikanan & energi terbarukan",
        "dinas_keywords": [
            # ── Pendidikan ──
            "PENDIDIKAN", "SEKOLAH", "KEBUDAYAAN",
            "PAUD", "DIKBUD", "DIKNAS",

            # ── Kesehatan ──
            "KESEHATAN", "RUMAH SAKIT", "RSUD", "PUSKESMAS",

            # ── Perikanan & Kelautan ──
            # Clue: Dinas Kelautan dan Perikanan, nelayan, 
            #        budidaya ikan, tangkap, pelabuhan perikanan
            "PERIKANAN", "KELAUTAN", "NELAYAN", "BUDIDAYA",
            "PELABUHAN PERIKANAN",

            # ── Energi dan Sumber Daya Mineral ──
            "ENERGI", "SUMBER DAYA MINERAL", "PERTAMBANGAN",
            "KELISTRIKAN", "LISTRIK", "TERBARUKAN",

            # ── Sosial ──
            "SOSIAL", "PEMBERDAYAAN MASYARAKAT",
        ],
        "produk_tsel": "Basic Connectivity, IoT Smart Connectivity, OmniChannel, IoT Smart Water Meter",
    },
}

print(f"✅ {len(ZONA_WILAYAH)} zona wilayah dikonfigurasi")
for z, info in ZONA_WILAYAH.items():
    print(f"   📍 {z}: {len(info['provinsi'])} provinsi, {len(info['dinas_keywords'])} keyword dinas")

✅ 6 zona wilayah dikonfigurasi
   📍 Sumatera: 10 provinsi, 32 keyword dinas
   📍 Kalimantan: 5 provinsi, 29 keyword dinas
   📍 Jawa: 6 provinsi, 24 keyword dinas
   📍 Bali — NusRa: 3 provinsi, 22 keyword dinas
   📍 Sulawesi: 6 provinsi, 24 keyword dinas
   📍 Papua — Maluku: 8 provinsi, 23 keyword dinas


### 1B. Pemerintah Pusat — Bidang Kenaikan Tertinggi

Keyword K/L (Kementerian/Lembaga) per bidang. Digunakan untuk filter kolom `Instansi_Pembeli`.

**Cara kerja filter:**
- Kolom `Instansi_Pembeli` di-match dengan keyword K/L relevan
- K/L yang **TIDAK mengandung** keyword → **dihapus** (misal Kemenpora di bidang Kesehatan)

In [3]:
# =============================================================================
# BIDANG FOKUS — KEYWORD K/L LENGKAP (TANPA SINGKATAN, TANPA PAGU)
# =============================================================================

BIDANG_FOKUS = {
    "Pendidikan": {
        "kl_keywords": [
            # ── Kementerian Pendidikan ──
            # Clue: Kemendikbud, Kemendikbudristek, Kemendikti,
            #        Kementerian Pendidikan Dasar dan Menengah,
            #        Kementerian Pendidikan Tinggi Sains dan Teknologi
            "KEMENTERIAN PENDIDIKAN",
            "PENDIDIKAN DASAR", "PENDIDIKAN MENENGAH",
            "PENDIDIKAN TINGGI", "SAINS DAN TEKNOLOGI",
            "KEBUDAYAAN",

            # ── Kementerian Agama (madrasah, pesantren, pendidikan agama) ──
            # Clue: Kementerian Agama, Kanwil Kemenag, 
            #        Madrasah Aliyah Negeri, MIN, MTsN, MAN
            "KEMENTERIAN AGAMA",

            # ── Universitas, Politeknik, dll ──
            # Clue: Universitas Negeri, Politeknik Negeri, 
            #        Institut Teknologi, Sekolah Tinggi Negeri, Akademi
            "UNIVERSITAS", "POLITEKNIK", "SEKOLAH TINGGI",
            "INSTITUT TEKNOLOGI", "INSTITUT PERTANIAN",
            "INSTITUT SENI", "AKADEMI",
            "MADRASAH", "PESANTREN",

            # ── Lembaga Riset & Ilmu Pengetahuan ──
            # Clue: BRIN, LIPI, Badan Riset dan Inovasi Nasional,
            #        Perpustakaan Nasional
            "BADAN RISET", "LEMBAGA ILMU PENGETAHUAN",
            "PERPUSTAKAAN NASIONAL",
        ],
        "kenaikan_label": "+11,6%",
    },

    "Pertahanan": {
        "kl_keywords": [
            # ── Kementerian Pertahanan ──
            # Clue: Kementerian Pertahanan, Kemhan, Sekretariat Jenderal Kemhan
            "KEMENTERIAN PERTAHANAN",

            # ── Tentara Nasional Indonesia (TNI) ──
            # Clue: Markas Besar TNI, TNI Angkatan Darat, TNI Angkatan Laut,
            #        TNI Angkatan Udara, Kodam, Korem, Kodim, Kopassus,
            #        Kostrad, Koarmabar, Koarmatim, Lanud, Lanal, Pangkalan
            "TENTARA NASIONAL", "ANGKATAN DARAT", "ANGKATAN LAUT",
            "ANGKATAN UDARA", "MARKAS BESAR",
            "KODAM", "KOREM", "KODIM", "KOPASSUS", "KOSTRAD",
            "KOARMABAR", "KOARMATIM", "KOLINLAMIL",
            "LANUD", "LANAL", "PANGKALAN",
            "DENMA", "DENMATRA", "BEKANGDAM",
            "RUMKIT", "RUMAH SAKIT TENTARA",

            # ── Kepolisian Negara Republik Indonesia ──
            # Clue: Kepolisian Negara, Polda, Polres, Polsek,
            #        Brimob, Densus, Sabhara, Reskrim
            "KEPOLISIAN NEGARA", "POLDA", "POLRES", "POLSEK",
            "BRIMOB", "DENSUS", "SABHARA",

            # ── Lembaga Intelijen & Pertahanan ──
            # Clue: Badan Intelijen Negara, BIN, BAIS,
            #        Badan Nasional Penanggulangan Terorisme, BNPT
            "BADAN INTELIJEN", "BAIS",
            "PENANGGULANGAN TERORISME",
        ],
        "kenaikan_label": "+47,5%",
    },

    "Kesehatan dan Perlindungan Sosial": {
        "kl_keywords": [
            # ── Kementerian Kesehatan ──
            # Clue: Kementerian Kesehatan, Kemenkes, 
            #        Direktorat Jenderal Pelayanan Kesehatan,
            #        Direktorat Jenderal Pencegahan dan Pengendalian Penyakit
            "KEMENTERIAN KESEHATAN",

            # ── Rumah Sakit Pusat & Fasilitas Kesehatan ──
            # Clue: RSUP, RSUPN, Rumah Sakit Umum Pusat,
            #        RS Jantung Harapan Kita, RS Kanker Dharmais, dll
            "RUMAH SAKIT UMUM PUSAT", "RSUP", "RSUPN",
            "RUMAH SAKIT", "RS JANTUNG", "RS KANKER",
            "RS PUSAT OTAK",

            # ── Badan Pengawas Obat dan Makanan ──
            # Clue: BPOM, Badan POM, Balai Besar POM
            "BADAN PENGAWAS OBAT", "BPOM",

            # ── Kependudukan & Keluarga Berencana ──
            # Clue: BKKBN, Badan Kependudukan, Keluarga Berencana
            "BADAN KEPENDUDUKAN", "KELUARGA BERENCANA",

            # ── Kementerian Sosial ──
            # Clue: Kementerian Sosial, Kemensos, 
            #        Perlindungan Sosial, Rehabilitasi Sosial,
            #        Jaminan Sosial, PKH
            "KEMENTERIAN SOSIAL",
            "PERLINDUNGAN SOSIAL", "REHABILITASI SOSIAL",
            "PEMBERDAYAAN SOSIAL",

            # ── BPJS ──
            # Clue: BPJS Kesehatan, BPJS Ketenagakerjaan,
            #        Badan Penyelenggara Jaminan Sosial
            "BADAN PENYELENGGARA JAMINAN", "BPJS",
        ],
        "kenaikan_label": "+8,6%",
    },

    "Subsidi Energi dan Non-Energi": {
        "kl_keywords": [
            # ── Kementerian Energi dan Sumber Daya Mineral ──
            # Clue: Kementerian ESDM, Dirjen Minyak dan Gas,
            #        Dirjen Ketenagalistrikan, Dirjen Mineral dan Batubara,
            #        Dirjen Energi Baru Terbarukan, Badan Geologi
            "KEMENTERIAN ENERGI", "SUMBER DAYA MINERAL",
            "MINYAK DAN GAS", "KETENAGALISTRIKAN",
            "MINERAL DAN BATUBARA", "ENERGI BARU",
            "ENERGI TERBARUKAN", "BADAN GEOLOGI",

            # ── Badan Pengatur Hilir Minyak dan Gas Bumi ──
            # Clue: BPH Migas, Badan Pengatur Hilir
            "BADAN PENGATUR HILIR", "MIGAS",

            # ── Kementerian Keuangan (terkait subsidi) ──
            # Clue: Kementerian Keuangan, Dirjen Anggaran,
            #        Badan Kebijakan Fiskal, Dirjen Perbendaharaan
            "KEMENTERIAN KEUANGAN",
            "DIREKTORAT JENDERAL ANGGARAN",
            "BADAN KEBIJAKAN FISKAL",
            "PERBENDAHARAAN",

            # ── Subsidi terkait lainnya ──
            # Clue: Pupuk Indonesia, Bulog, Perum Bulog
            "PUPUK INDONESIA", "BULOG",
        ],
        "kenaikan_label": "+10,7%",
    },

    "Pembangunan Ekonomi dan Infrastruktur": {
        "kl_keywords": [
            # ── Kementerian Pekerjaan Umum dan Perumahan Rakyat ──
            # Clue: Kementerian PUPR, Dirjen Bina Marga, Dirjen Cipta Karya,
            #        Dirjen Sumber Daya Air, Dirjen Perumahan,
            #        Balai Besar Pelaksanaan Jalan Nasional
            "KEMENTERIAN PEKERJAAN UMUM", "PERUMAHAN RAKYAT",
            "BINA MARGA", "CIPTA KARYA", "SUMBER DAYA AIR",
            "BALAI BESAR PELAKSANAAN JALAN",
            "BALAI PELAKSANAAN JALAN",

            # ── Kementerian Perhubungan ──
            # Clue: Kementerian Perhubungan, Dirjen Perhubungan Darat,
            #        Dirjen Perhubungan Laut, Dirjen Perhubungan Udara,
            #        Dirjen Perkeretaapian, LPPNPI (AirNav)
            "KEMENTERIAN PERHUBUNGAN",
            "PERHUBUNGAN DARAT", "PERHUBUNGAN LAUT",
            "PERHUBUNGAN UDARA", "PERKERETAAPIAN",
            "NAVIGASI PENERBANGAN",

            # ── Kementerian Perindustrian ──
            # Clue: Kementerian Perindustrian, Kemenperin, 
            #        Balai Besar Industri, Balai Riset Industri
            "KEMENTERIAN PERINDUSTRIAN",

            # ── Kementerian Perdagangan ──
            # Clue: Kementerian Perdagangan, Kemendag,
            #        Dirjen Perdagangan Dalam Negeri, Dirjen Perdagangan Luar Negeri
            "KEMENTERIAN PERDAGANGAN",

            # ── Kementerian Koperasi dan UKM ──
            # Clue: Kementerian Koperasi, Kemenkop,
            #        Usaha Kecil dan Menengah
            "KEMENTERIAN KOPERASI",
            "USAHA KECIL", "USAHA MENENGAH",

            # ── Bappenas / Kementerian PPN ──
            # Clue: Badan Perencanaan Pembangunan Nasional, Bappenas,
            #        Kementerian Perencanaan Pembangunan
            "BADAN PERENCANAAN PEMBANGUNAN NASIONAL",
            "KEMENTERIAN PERENCANAAN",

            # ── Kementerian Investasi / BKPM ──
            # Clue: Kementerian Investasi, BKPM, 
            #        Badan Koordinasi Penanaman Modal
            "KEMENTERIAN INVESTASI",
            "BADAN KOORDINASI PENANAMAN MODAL",

            # ── Kementerian Agraria dan Tata Ruang / BPN ──
            # Clue: Kementerian Agraria, Badan Pertanahan Nasional,
            #        Tata Ruang, Kantor Pertanahan
            "KEMENTERIAN AGRARIA", "TATA RUANG",
            "BADAN PERTANAHAN NASIONAL", "KANTOR PERTANAHAN",
        ],
        "kenaikan_label": "+50,4%",
    },
}

print(f"\n✅ {len(BIDANG_FOKUS)} bidang fokus dikonfigurasi")
for b, info in BIDANG_FOKUS.items():
    print(f"   📈 {b} ({info['kenaikan_label']}): {len(info['kl_keywords'])} keyword K/L")


✅ 5 bidang fokus dikonfigurasi
   📈 Pendidikan (+11,6%): 19 keyword K/L
   📈 Pertahanan (+47,5%): 32 keyword K/L
   📈 Kesehatan dan Perlindungan Sosial (+8,6%): 18 keyword K/L
   📈 Subsidi Energi dan Non-Energi (+10,7%): 16 keyword K/L
   📈 Pembangunan Ekonomi dan Infrastruktur (+50,4%): 26 keyword K/L


### 1C. ICT Classification — Whitelist + Blacklist

**Whitelist:** Keyword yang menandakan paket ICT (internet, bandwidth, server, laptop, dll)

**Blacklist:** Keyword yang menandakan false positive (obat, vaksin, konstruksi, makanan, dll)

**Rule:** ICT = match whitelist AND NOT match blacklist

In [4]:
# =============================================================================
# ICT WHITELIST PATTERNS (dari notebook asli)
# =============================================================================

ICT_WHITELIST_PATTERNS = {
    'CONNECTIVITY': [
        (r'\bINTERNET\b', 'internet'), (r'\bBANDWIDTH\b', 'bandwidth'),
        (r'\bBROADBAND\b', 'broadband'), (r'\bFIBER\s*OPTI[CK]\b', 'fiber optic'),
        (r'\bFIBER\b(?!\s*GLASS)', 'fiber'), (r'\bMPLS\b', 'mpls'),
        (r'\bVPN\b', 'vpn'), (r'\bVSAT\b', 'vsat'), (r'\bWI[\s\-]?FI\b', 'wifi'),
        (r'\bWIFI\b', 'wifi'), (r'\bWIRELESS\b', 'wireless'), (r'\bHOTSPOT\b', 'hotspot'),
    ],
    'CLOUD_DATACENTER': [
        (r'\bCLOUD\b(?!\s*NINE)', 'cloud'), (r'\bDATA\s*CENTER\b', 'data center'),
        (r'\bCOLOCATION\b', 'colocation'), (r'\bHOSTING\b', 'hosting'),
        (r'\bVIRTUAL\s+SERVER\b', 'virtual server'), (r'\bVPS\b', 'vps'),
    ],
    'TELECOM': [
        (r'\bPULSA\b', 'pulsa'), (r'\bPAKET\s+DATA\b', 'paket data'),
        (r'\bSIM\s*CARD\b', 'sim card'), (r'\bTELEKOMUNIKASI\b', 'telekomunikasi'),
        (r'\bPABX\b', 'pabx'), (r'\bVOIP\b', 'voip'), (r'\bIP\s+PHONE\b', 'ip phone'),
        (r'\bCALL\s+CENTER\b', 'call center'),
    ],
    'COLLABORATION': [
        (r'\bVIDEO\s*CONFERENCE\b', 'video conference'), (r'\bZOOM\b(?!\s*IN|\s*OUT)', 'zoom'),
        (r'\bWEBINAR\b', 'webinar'), (r'\bMICROSOFT\s+TEAMS\b', 'ms teams'),
    ],
    'IOT_SMART': [
        (r'\bIOT\b', 'iot'), (r'\bGPS\s+TRACK(ER|ING)\b', 'gps tracking'),
        (r'\bTELEMATIC[S]?\b', 'telematics'), (r'\bSMART\s+CITY\b', 'smart city'),
    ],
    'SURVEILLANCE': [
        (r'\bCCTV\b', 'cctv'), (r'\bSURVEILLANCE\b', 'surveillance'),
        (r'\bIP\s+CAMERA\b', 'ip camera'), (r'\bNVR\b', 'nvr'), (r'\bDVR\b(?!\s+PLAYER)', 'dvr'),
        (r'\bACCESS\s+CONTROL\b', 'access control'), (r'\bBIOMETRIC\b', 'biometric'),
    ],
    'HARDWARE_COMPUTER': [
        (r'\bKOMPUTER\b', 'komputer'), (r'\bCOMPUTER\b', 'computer'),
        (r'\bLAPTOP\b', 'laptop'), (r'\bNOTEBOOK\b', 'notebook'),
        (r'\bDESKTOP\b', 'desktop'), (r'\bWORKSTATION\b', 'workstation'),
    ],
    'HARDWARE_SERVER': [
        (r'\bSERVER\b(?!\s+MAKANAN|\s+MINUMAN)', 'server'),
        (r'\bSTORAGE\b(?!\s+BOX|\s+RACK\s+BESI)', 'storage'),
        (r'\bRACK\s+SERVER\b', 'rack server'), (r'\bUPS\b(?!\s+DELIVERY)', 'ups'),
    ],
    'HARDWARE_NETWORK': [
        (r'\bROUTER\b', 'router'), (r'\bSWITCH\b(?!\s+ON|\s+OFF)', 'switch'),
        (r'\bFIREWALL\b', 'firewall'), (r'\bMODEM\b', 'modem'),
    ],
    'SOFTWARE': [
        (r'\bSOFTWARE\b', 'software'), (r'\bAPLIKASI\b(?!\s+LAMARAN)', 'aplikasi'),
        (r'\bSISTEM\s+INFORMASI\b', 'sistem informasi'), (r'\bWEBSITE\b', 'website'),
        (r'\bDATABASE\b', 'database'), (r'\bERP\b', 'erp'), (r'\bANTIVIRUS\b', 'antivirus'),
        (r'\bLISENSI\b', 'lisensi'), (r'\bLICENSE\b', 'license'),
    ],
    'IT_SERVICES': [
        (r'\bMAINTENANCE\s+(JARINGAN|SERVER|IT|NETWORK)\b', 'maintenance IT'),
        (r'\bSYSTEM\s+INTEGRAT(OR|ION)\b', 'system integrator'),
        (r'\bMANAGED\s+SERVICE\b', 'managed service'),
    ],
    'SECURITY': [
        (r'\bCYBER\s*SECURITY\b', 'cyber security'), (r'\bNETWORK\s+SECURITY\b', 'network security'),
    ],
}

PACKAGE_BLACKLIST_PATTERNS = [
    r'\bBUKU\b', r'\bPRINTER\b', r'\bTONER\b', r'\bBANGUNAN\b', r'\bKONSTRUKSI\b', r'\bTINTA\b',
    r'\bOBAT\b', r'\bVAKSIN\b', r'\bALAT\s+KESEHATAN\b', r'\bMEDIS\b',
    r'\bELEKTROMEDI[CKS]?\b', r'\bPATIENT\s+MONITOR\b', r'\bVENTILATOR\b',
    r'\bINCUBATOR\b', r'\bENDOSCOP[EY]\b', r'\bCT\s+SCAN\b', r'\bMRI\b', r'\bUSG\b',
    r'\bMAKANAN\b', r'\bMINUMAN\b', r'\bKATERING\b', r'\bATK\b', r'\bSERAGAM\b',
    r'\bMOBIL\b(?!\s+APP)', r'\bKENDARAAN\b',
]

# Compile regex
_all_ict_rx = []
for pats in ICT_WHITELIST_PATTERNS.values():
    _all_ict_rx.extend([p[0] for p in pats])
_ict_re = re.compile("|".join(_all_ict_rx), re.IGNORECASE)
_bl_re = re.compile("|".join(PACKAGE_BLACKLIST_PATTERNS), re.IGNORECASE)
_cat_re = {c: re.compile("|".join([p[0] for p in pats]), re.IGNORECASE)
            for c, pats in ICT_WHITELIST_PATTERNS.items()}

def classify_ict(text):
    """True = ICT, False = Non-ICT"""
    if pd.isna(text): return False
    s = str(text)
    return bool(_ict_re.search(s) and not _bl_re.search(s))

def get_ict_category(text):
    """Return ICT category or None"""
    if pd.isna(text): return None
    s = str(text)
    if _bl_re.search(s): return None
    for cat, rx in _cat_re.items():
        if rx.search(s): return cat
    return None

print(f"\n✅ ICT Whitelist: {len(ICT_WHITELIST_PATTERNS)} kategori, {len(_all_ict_rx)} patterns")
print(f"✅ Blacklist: {len(PACKAGE_BLACKLIST_PATTERNS)} patterns")


✅ ICT Whitelist: 12 kategori, 69 patterns
✅ Blacklist: 25 patterns


---
## 2. LOAD DATA

In [5]:
# =============================================================================
# LOAD DATA — dari SQLite atau CSV
# =============================================================================

# ── GANTI PATH SESUAI LOKASI FILE KAMU ──
DB_PATH = "Datamart_Final_Report.db"
# CSV_PATH = "contoh_data.csv"   # alternatif jika pakai CSV

if os.path.exists(DB_PATH):
    conn = sqlite3.connect(DB_PATH)
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
    print(f"📂 Tables in DB: {tables['name'].tolist()}")
    
    # Auto-detect table
    tbl = "Datamart_Final_Report"
    if tbl not in tables["name"].values:
        tbl = tables.iloc[0]["name"]
    
    df_raw = pd.read_sql(f"SELECT * FROM [{tbl}]", conn)
    conn.close()
    print(f"✅ Loaded {len(df_raw):,} rows from SQLite [{tbl}]")
else:
    # Fallback ke CSV
    csv_candidates = [f for f in os.listdir(".") if f.endswith(".csv")]
    if csv_candidates:
        csv_path = csv_candidates[0]
        df_raw = pd.read_csv(csv_path)
        print(f"✅ Loaded {len(df_raw):,} rows from CSV [{csv_path}]")
    else:
        raise FileNotFoundError(f"❌ {DB_PATH} tidak ditemukan dan tidak ada CSV di folder ini")

print(f"\n📋 Kolom: {df_raw.columns.tolist()}")
print(f"📋 Shape: {df_raw.shape}")
df_raw.head(3)

📂 Tables in DB: ['datamart']
✅ Loaded 6,149,392 rows from SQLite [datamart]

📋 Kolom: ['ID_RUP', 'Nama_Paket', 'Kategori_Paket', 'Pagu_Rp', 'Jenis_Usaha', 'Metode_Pemilihan', 'Jenis_Pengadaan', 'Tanggal_Pemilihan', 'Instansi_Pembeli', 'Satuan_Kerja', 'Lokasi', 'Nama_Pemenang', 'Nama_Produk', 'Kategori_Produk', 'Kuantitas_Produk', 'Nama_Manufaktur', 'Total_Pelaksanaan_Rp', 'Sumber_Data', 'Updated_At']
📋 Shape: (6149392, 19)


,ID_RUP,Nama_Paket,Kategori_Paket,Pagu_Rp,Jenis_Usaha,Metode_Pemilihan,Jenis_Pengadaan,Tanggal_Pemilihan,Instansi_Pembeli,Satuan_Kerja,Lokasi,Nama_Pemenang,Nama_Produk,Kategori_Produk,Kuantitas_Produk,Nama_Manufaktur,Total_Pelaksanaan_Rp,Sumber_Data,Updated_At
0,53647021,Pembayaran Gaji PJLP Ruang Interaksi Warga/RPTRA,Others,812415744.0,Kecil,Pengadaan Langsung,Jasa Lainnya,December 2024,Provinsi DKI Jakarta,KELURAHAN CILANGKAP - JAKTIM,"DKI Jakarta, Jakarta Timur (Kota)",A**** B*******,,,,,701578930.0,data inaproc,2026-02-04 13:00:00
1,53682877,Penyediaan Jasa Petugas Penanganan Prasarana d...,,42866472623.0,Kecil,Pengadaan Langsung,Jasa Lainnya,December 2024,Provinsi DKI Jakarta,SUKU DINAS PERTAMANAN DAN HUTAN KOTA KOTA ADMI...,"DKI Jakarta, Jakarta Pusat (Kota)",K*** H*******,,,,,701578930.0,data inaproc,2026-02-04 13:00:00
2,53863946,Penyedia Jasa Lainnya Petugas Pengolahan Sampa...,,7012418400.0,Kecil,Pengadaan Langsung,Jasa Lainnya,December 2024,Provinsi DKI Jakarta,SUKU DINAS LINGKUNGAN HIDUP KOTA ADMINISTRASI ...,"DKI Jakarta, Jakarta Utara (Kota)",P**** t**** i******,,,,,88560225.0,bigbox,2026-01-24 14:00:00


---
## 3. DATA PREPROCESSING

In [6]:
# =============================================================================
# PREPROCESSING
# =============================================================================

df = df_raw.copy()

# ── Parse numerik ──
for col in ["Pagu_Rp", "Total_Pelaksanaan_Rp"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# ── FILTER: Nama_Pemenang WAJIB TERISI ──
n_before = len(df)
df = df[df["Nama_Pemenang"].notna() & (df["Nama_Pemenang"].str.strip() != "")]
print(f"🔍 Filter Nama_Pemenang terisi: {n_before:,} → {len(df):,} ({n_before - len(df):,} dihapus)")

# ── Extract Provinsi dari Lokasi ──
def extract_provinsi(lokasi):
    """'Sulawesi Tenggara, Kolaka Utara (Kab.)' → 'Sulawesi Tenggara'"""
    if pd.isna(lokasi): return "Lainnya"
    return str(lokasi).split("|")[0].split(",")[0].strip()

df["Provinsi"] = df["Lokasi"].apply(extract_provinsi)

# ── Map Zona Wilayah ──
def get_zona(prov):
    for zona, info in ZONA_WILAYAH.items():
        if prov in info["provinsi"]:
            return zona
    return "Lainnya"

df["Zona"] = df["Provinsi"].apply(get_zona)

# ── ICT Classification ──
df["Is_ICT"] = df["Nama_Paket"].apply(classify_ict)
df["Kategori_ICT"] = df["Nama_Paket"].apply(get_ict_category)
df["Sektor"] = df["Is_ICT"].map({True: "ICT", False: "Non-ICT"})

print(f"\n📊 Total data (Nama_Pemenang terisi): {len(df):,}")
print(f"📊 ICT: {df['Is_ICT'].sum():,} | Non-ICT: {(~df['Is_ICT']).sum():,}")
print(f"\n📊 Distribusi Zona:")
print(df["Zona"].value_counts().to_string())
print(f"\n📊 Distribusi Provinsi (top 10):")
print(df["Provinsi"].value_counts().head(10).to_string())

🔍 Filter Nama_Pemenang terisi: 6,149,392 → 872,669 (5,276,723 dihapus)

📊 Total data (Nama_Pemenang terisi): 872,669
📊 ICT: 52,425 | Non-ICT: 820,244

📊 Distribusi Zona:
Zona
Jawa              406362
Sumatera          159911
Kalimantan        147774
Sulawesi           75162
Bali — NusRa       49737
Papua — Maluku     26633
Lainnya             7090

📊 Distribusi Provinsi (top 10):
Provinsi
Jawa Timur            126169
Jawa Barat            101355
Jawa Tengah            73463
DKI Jakarta            57605
Kalimantan Selatan     44532
Kalimantan Timur       42925
Banten                 31853
Sulawesi Selatan       28442
Bali                   27820
Lampung                25986


In [7]:
# Helper: Match keyword ke text (case insensitive)
def match_keywords(text, keywords):
    """Cek apakah text mengandung salah satu keyword."""
    if pd.isna(text): return False
    up = str(text).upper()
    return any(kw.upper() in up for kw in keywords)

# Helper: Top 20 SI Channel dengan min satker
def get_top20_si(df_input, min_satker=5, top_n=20):
    """Aggregate pemenang, filter ≥min_satker, ambil top N."""
    if len(df_input) == 0:
        return pd.DataFrame()
    
    agg = (df_input.groupby("Nama_Pemenang")
           .agg(
               Total_Pagu_Rp=("Pagu_Rp", "sum"),
               Jumlah_Paket=("Pagu_Rp", "count"),
               Jumlah_Satuan_Kerja=("Satuan_Kerja", "nunique"),
               Jumlah_Instansi=("Instansi_Pembeli", "nunique"),
               Jumlah_Lokasi=("Lokasi", "nunique"),
               Contoh_Paket=("Nama_Paket", "first"),
               Contoh_Satker=("Satuan_Kerja", "first"),
               Contoh_Instansi=("Instansi_Pembeli", "first"),
           )
           .reset_index())
    
    agg = agg[agg["Jumlah_Satuan_Kerja"] >= min_satker]
    agg = agg.sort_values("Total_Pagu_Rp", ascending=False).head(top_n)
    agg.index = range(1, len(agg) + 1)
    agg.index.name = "Rank"
    return agg

# Helper: Format Rupiah
def fmt_rp(v):
    if pd.isna(v) or v == 0: return "Rp 0"
    a = abs(v)
    if a >= 1e12: return f"Rp {v/1e12:,.2f} T"
    if a >= 1e9:  return f"Rp {v/1e9:,.2f} M"
    if a >= 1e6:  return f"Rp {v/1e6:,.1f} Jt"
    return f"Rp {v:,.0f}"

print("✅ Helper functions ready")

✅ Helper functions ready


---
## 4. ANALISIS 1 — PEMERINTAH DAERAH (Per Zona Wilayah)

Filter: 
- Lokasi → Provinsi → Zona
- Satuan_Kerja HARUS mengandung keyword dinas relevan (yang tidak relevan dihapus)
- Nama_Pemenang terisi
- Top 20 pemenang ≥5 Satuan Kerja

In [8]:
# =============================================================================
# ANALISIS PER ZONA WILAYAH — PEMERINTAH DAERAH
# =============================================================================

MIN_SATKER = 5
OUTPUT_DIR = "output_si_channel"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 70)
print("  BAGIAN 1: PEMERINTAH DAERAH — SI CHANNEL PER ZONA WILAYAH")
print("=" * 70)

all_zona_results = {}

for zona_name, zona_info in ZONA_WILAYAH.items():
    print(f"\n{'─' * 70}")
    print(f"📍 ZONA: {zona_name}")
    print(f"   Fokus: {zona_info['fokus']}")
    print(f"   Provinsi: {', '.join(zona_info['provinsi'])}")
    print(f"   Keyword Dinas: {', '.join(zona_info['dinas_keywords'][:8])}...")
    print(f"   Produk Telkomsel: {zona_info['produk_tsel']}")
    print(f"{'─' * 70}")
    
    # ── Step 1: Filter by zona (provinsi) ──
    df_zona = df[df["Zona"] == zona_name].copy()
    print(f"   [1] Data di zona {zona_name}: {len(df_zona):,} paket")
    
    if len(df_zona) == 0:
        print(f"   ⚠️ Tidak ada data untuk zona {zona_name}")
        continue
    
    # ── Step 2: Filter Satuan_Kerja by keyword dinas relevan ──
    # Match: Satuan_Kerja ATAU Instansi_Pembeli mengandung keyword
    mask_satker = df_zona["Satuan_Kerja"].apply(
        lambda x: match_keywords(x, zona_info["dinas_keywords"]))
    mask_instansi = df_zona["Instansi_Pembeli"].apply(
        lambda x: match_keywords(x, zona_info["dinas_keywords"]))
    mask_relevan = mask_satker | mask_instansi
    
    df_zona_filtered = df_zona[mask_relevan].copy()
    n_removed = len(df_zona) - len(df_zona_filtered)
    print(f"   [2] Filter dinas relevan: {len(df_zona):,} → {len(df_zona_filtered):,} ({n_removed:,} dihapus)")
    
    if len(df_zona_filtered) == 0:
        print(f"   ⚠️ Tidak ada data setelah filter dinas")
        continue
    
    # ── Step 3: Top 20 ICT ──
    df_ict = df_zona_filtered[df_zona_filtered["Is_ICT"] == True]
    top_ict = get_top20_si(df_ict, min_satker=MIN_SATKER)
    print(f"\n   💻 ICT: {len(df_ict):,} paket → Top 20 SI (≥{MIN_SATKER} satker): {len(top_ict)} pemenang")
    if len(top_ict) > 0:
        for _, row in top_ict.head(5).iterrows():
            print(f"      #{_}: {row['Nama_Pemenang'][:40]:<40} | {fmt_rp(row['Total_Pagu_Rp'])} | {row['Jumlah_Satuan_Kerja']} satker")
    
    # ── Step 4: Top 20 Non-ICT ──
    df_non = df_zona_filtered[df_zona_filtered["Is_ICT"] == False]
    top_non = get_top20_si(df_non, min_satker=MIN_SATKER)
    print(f"\n   📦 Non-ICT: {len(df_non):,} paket → Top 20 SI (≥{MIN_SATKER} satker): {len(top_non)} pemenang")
    if len(top_non) > 0:
        for _, row in top_non.head(5).iterrows():
            print(f"      #{_}: {row['Nama_Pemenang'][:40]:<40} | {fmt_rp(row['Total_Pagu_Rp'])} | {row['Jumlah_Satuan_Kerja']} satker")
    
    # ── Step 5: Export CSV ──
    zona_safe = zona_name.replace(" ", "_").replace("—", "-")
    
    if len(top_ict) > 0:
        fn = f"{OUTPUT_DIR}/SI_ICT_{zona_safe}.csv"
        top_ict.to_csv(fn)
        print(f"\n   📥 Saved: {fn}")
    
    if len(top_non) > 0:
        fn = f"{OUTPUT_DIR}/SI_NonICT_{zona_safe}.csv"
        top_non.to_csv(fn)
        print(f"   📥 Saved: {fn}")
    
    # Export raw data juga
    if len(df_zona_filtered) > 0:
        fn = f"{OUTPUT_DIR}/RAW_{zona_safe}_filtered.csv"
        df_zona_filtered.to_csv(fn, index=False)
        print(f"   📥 Saved: {fn} ({len(df_zona_filtered):,} rows)")
    
    all_zona_results[zona_name] = {
        "total_paket": len(df_zona_filtered),
        "ict_paket": len(df_ict),
        "non_ict_paket": len(df_non),
        "top_ict": top_ict,
        "top_non": top_non,
    }

print(f"\n{'=' * 70}")
print(f"✅ Analisis Zona Wilayah selesai — {len(all_zona_results)} zona diproses")

  BAGIAN 1: PEMERINTAH DAERAH — SI CHANNEL PER ZONA WILAYAH

──────────────────────────────────────────────────────────────────────
📍 ZONA: Sumatera
   Fokus: Revitalisasi sekolah, irigasi, dan koperasi lokal
   Provinsi: Aceh, Sumatera Utara, Sumatera Barat, Riau, Kepulauan Riau, Jambi, Sumatera Selatan, Bangka Belitung, Bengkulu, Lampung
   Keyword Dinas: PENDIDIKAN, SEKOLAH, KEBUDAYAAN, PAUD, TAMAN KANAK, DIKBUD, DIKNAS, DIKDASMEN...
   Produk Telkomsel: IoT Smart Farming, Fleet Management, Telkomsel Learning Platform, IoT Smart Water Meter
──────────────────────────────────────────────────────────────────────
   [1] Data di zona Sumatera: 159,911 paket
   [2] Filter dinas relevan: 159,911 → 63,311 (96,600 dihapus)

   💻 ICT: 1,928 paket → Top 20 SI (≥5 satker): 11 pemenang
      #1: AGRES BATAM TEKNOLOGI                    | Rp 1.77 M | 7 satker
      #2: INTER MEDIALINK SOLUSI                   | Rp 1.32 M | 8 satker
      #3: PT. MULTI DATA PALEMBANG                 | Rp 1.02 M |

---
## 5. ANALISIS 2 — PEMERINTAH PUSAT (Per Bidang Kenaikan Tertinggi)

Filter:
- Instansi_Pembeli HARUS mengandung keyword K/L relevan (yang tidak relevan dihapus)
- Nama_Pemenang terisi
- Top 20 pemenang ≥5 Satuan Kerja

In [9]:
# =============================================================================
# ANALISIS PER BIDANG — PEMERINTAH PUSAT (KEMENTERIAN & LEMBAGA)
# =============================================================================

print("=" * 70)
print("  BAGIAN 2: PEMERINTAH PUSAT — SI CHANNEL PER BIDANG")
print("=" * 70)

all_bidang_results = {}

for bidang_name, bidang_info in BIDANG_FOKUS.items():
    print(f"\n{'─' * 70}")
    print(f"📈 BIDANG: {bidang_name} (Kenaikan {bidang_info['kenaikan_label']})")
    print(f"   Keyword K/L: {', '.join(bidang_info['kl_keywords'][:6])}...")
    print(f"{'─' * 70}")
    
    # ── Step 1: Filter Instansi_Pembeli by K/L keyword ──
    mask_instansi = df["Instansi_Pembeli"].apply(
        lambda x: match_keywords(x, bidang_info["kl_keywords"]))
    
    # Juga cek Satuan_Kerja (untuk tangkap satker seperti "Rumkit TNI AD" dll)
    mask_satker = df["Satuan_Kerja"].apply(
        lambda x: match_keywords(x, bidang_info["kl_keywords"]))
    
    mask_bidang = mask_instansi | mask_satker
    df_bidang = df[mask_bidang].copy()
    
    print(f"   [1] Filter K/L relevan: {len(df):,} total → {len(df_bidang):,} paket bidang {bidang_name}")
    
    if len(df_bidang) == 0:
        print(f"   ⚠️ Tidak ada data untuk bidang {bidang_name}")
        continue
    
    # ── Step 2: Show top Instansi ──
    top_inst = (df_bidang.groupby("Instansi_Pembeli")
                .agg(Pagu=("Pagu_Rp", "sum"), Paket=("Pagu_Rp", "count"))
                .sort_values("Pagu", ascending=False).head(10))
    print(f"\n   🏛️ Top 10 Instansi Pembeli:")
    for inst, row in top_inst.iterrows():
        print(f"      {str(inst)[:50]:<50} | {fmt_rp(row['Pagu'])} | {row['Paket']:,} paket")
    
    # ── Step 3: Top 20 ICT ──
    df_ict = df_bidang[df_bidang["Is_ICT"] == True]
    top_ict = get_top20_si(df_ict, min_satker=MIN_SATKER)
    print(f"\n   💻 ICT: {len(df_ict):,} paket → Top 20 SI (≥{MIN_SATKER} satker): {len(top_ict)} pemenang")
    if len(top_ict) > 0:
        for _, row in top_ict.head(5).iterrows():
            print(f"      #{_}: {row['Nama_Pemenang'][:40]:<40} | {fmt_rp(row['Total_Pagu_Rp'])} | {row['Jumlah_Satuan_Kerja']} satker")
    
    # ── Step 4: Top 20 Non-ICT ──
    df_non = df_bidang[df_bidang["Is_ICT"] == False]
    top_non = get_top20_si(df_non, min_satker=MIN_SATKER)
    print(f"\n   📦 Non-ICT: {len(df_non):,} paket → Top 20 SI (≥{MIN_SATKER} satker): {len(top_non)} pemenang")
    if len(top_non) > 0:
        for _, row in top_non.head(5).iterrows():
            print(f"      #{_}: {row['Nama_Pemenang'][:40]:<40} | {fmt_rp(row['Total_Pagu_Rp'])} | {row['Jumlah_Satuan_Kerja']} satker")
    
    # ── Step 5: Export CSV ──
    bidang_safe = bidang_name.replace(" ", "_").replace("&", "dan")
    
    if len(top_ict) > 0:
        fn = f"{OUTPUT_DIR}/SI_ICT_Bidang_{bidang_safe}.csv"
        top_ict.to_csv(fn)
        print(f"\n   📥 Saved: {fn}")
    
    if len(top_non) > 0:
        fn = f"{OUTPUT_DIR}/SI_NonICT_Bidang_{bidang_safe}.csv"
        top_non.to_csv(fn)
        print(f"   📥 Saved: {fn}")
    
    if len(df_bidang) > 0:
        fn = f"{OUTPUT_DIR}/RAW_Bidang_{bidang_safe}_filtered.csv"
        df_bidang.to_csv(fn, index=False)
        print(f"   📥 Saved: {fn} ({len(df_bidang):,} rows)")
    
    all_bidang_results[bidang_name] = {
        "total_paket": len(df_bidang),
        "ict_paket": len(df_ict),
        "non_ict_paket": len(df_non),
        "top_ict": top_ict,
        "top_non": top_non,
    }

print(f"\n{'=' * 70}")
print(f"✅ Analisis Bidang selesai — {len(all_bidang_results)} bidang diproses")

  BAGIAN 2: PEMERINTAH PUSAT — SI CHANNEL PER BIDANG

──────────────────────────────────────────────────────────────────────
📈 BIDANG: Pendidikan (Kenaikan +11,6%)
   Keyword K/L: KEMENTERIAN PENDIDIKAN, PENDIDIKAN DASAR, PENDIDIKAN MENENGAH, PENDIDIKAN TINGGI, SAINS DAN TEKNOLOGI, KEBUDAYAAN...
──────────────────────────────────────────────────────────────────────
   [1] Filter K/L relevan: 872,669 total → 59,166 paket bidang Pendidikan

   🏛️ Top 10 Instansi Pembeli:
      Kementerian Pendidikan Dasar dan Menengah          | Rp 4.49 T | 630.0 paket
      Kementerian Pendidikan Tinggi, Sains, dan Teknolog | Rp 3.74 T | 4,733.0 paket
      Kementerian Agama                                  | Rp 3.37 T | 4,437.0 paket
      Provinsi Kalimantan Timur                          | Rp 861.44 M | 1,721.0 paket
      Badan Riset dan Inovasi Nasional                   | Rp 727.05 M | 226.0 paket
      Provinsi Jawa Tengah                               | Rp 407.80 M | 601.0 paket
      Kementeria

---
## 6. RINGKASAN EKSEKUTIF

In [10]:
# =============================================================================
# RINGKASAN EKSEKUTIF
# =============================================================================

print("=" * 70)
print("  RINGKASAN EKSEKUTIF — SI CHANNEL POTENSIAL")
print("=" * 70)

print(f"\n📊 Total data (Nama_Pemenang terisi): {len(df):,} paket")
print(f"📊 Total Pagu: {fmt_rp(df['Pagu_Rp'].sum())}")
print(f"📊 ICT: {df['Is_ICT'].sum():,} paket | Non-ICT: {(~df['Is_ICT']).sum():,} paket")

print(f"\n{'─' * 70}")
print(f"  BAGIAN 1: PEMERINTAH DAERAH — PER ZONA WILAYAH")
print(f"{'─' * 70}")
for zona, res in all_zona_results.items():
    n_ict = len(res["top_ict"])
    n_non = len(res["top_non"])
    print(f"  📍 {zona:<20} | {res['total_paket']:>8,} paket | "
          f"SI ICT: {n_ict:>2} | SI Non-ICT: {n_non:>2}")

print(f"\n{'─' * 70}")
print(f"  BAGIAN 2: PEMERINTAH PUSAT — PER BIDANG KENAIKAN TERTINGGI")
print(f"{'─' * 70}")
for bidang, res in all_bidang_results.items():
    n_ict = len(res["top_ict"])
    n_non = len(res["top_non"])
    kn = BIDANG_FOKUS[bidang]["kenaikan_label"]
    print(f"  📈 {bidang:<45} ({kn:>6}) | {res['total_paket']:>8,} paket | "
          f"SI ICT: {n_ict:>2} | SI Non-ICT: {n_non:>2}")

print(f"\n📁 Semua file CSV tersimpan di folder: {OUTPUT_DIR}/")
files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith(".csv")]
for f in sorted(files):
    size = os.path.getsize(f"{OUTPUT_DIR}/{f}")
    print(f"   📄 {f} ({size/1024:.0f} KB)")

print(f"\n{'=' * 70}")
print(f"  SELESAI — {datetime.now():%d %B %Y %H:%M}")
print(f"{'=' * 70}")

  RINGKASAN EKSEKUTIF — SI CHANNEL POTENSIAL

📊 Total data (Nama_Pemenang terisi): 872,669 paket
📊 Total Pagu: Rp 406.38 T
📊 ICT: 52,425 paket | Non-ICT: 820,244 paket

──────────────────────────────────────────────────────────────────────
  BAGIAN 1: PEMERINTAH DAERAH — PER ZONA WILAYAH
──────────────────────────────────────────────────────────────────────
  📍 Sumatera             |   63,311 paket | SI ICT: 11 | SI Non-ICT: 20
  📍 Kalimantan           |   40,807 paket | SI ICT:  1 | SI Non-ICT: 20
  📍 Jawa                 |   30,181 paket | SI ICT: 20 | SI Non-ICT: 20
  📍 Bali — NusRa         |   15,666 paket | SI ICT: 11 | SI Non-ICT: 20
  📍 Sulawesi             |   27,071 paket | SI ICT:  9 | SI Non-ICT: 20
  📍 Papua — Maluku       |    8,442 paket | SI ICT:  0 | SI Non-ICT: 20

──────────────────────────────────────────────────────────────────────
  BAGIAN 2: PEMERINTAH PUSAT — PER BIDANG KENAIKAN TERTINGGI
──────────────────────────────────────────────────────────────────────
  📈 

---
## 7. DETAIL VIEW — Cek Salah Satu Hasil

Uncomment dan ganti nama zona/bidang untuk melihat detail

In [11]:
# =============================================================================
# QUICK VIEW — ZONA TERTENTU (uncomment sesuai kebutuhan)
# =============================================================================

# Contoh: Lihat Top ICT Jawa
# if "Jawa" in all_zona_results:
#     print("Top 20 SI ICT — Zona Jawa:")
#     display(all_zona_results["Jawa"]["top_ict"])

# Contoh: Lihat Top Non-ICT Bidang Pertahanan
# if "Pertahanan" in all_bidang_results:
#     print("Top 20 SI Non-ICT — Bidang Pertahanan:")
#     display(all_bidang_results["Pertahanan"]["top_non"])

# ── Tampilkan semua Top ICT per Zona ──
for zona, res in all_zona_results.items():
    if len(res["top_ict"]) > 0:
        print(f"\n{'═' * 70}")
        print(f"💻 TOP SI CHANNEL ICT — ZONA {zona.upper()}")
        print(f"{'═' * 70}")
        display_df = res["top_ict"][["Nama_Pemenang", "Total_Pagu_Rp", 
                                      "Jumlah_Paket", "Jumlah_Satuan_Kerja", 
                                      "Jumlah_Instansi"]].copy()
        display_df["Total_Pagu_Rp"] = display_df["Total_Pagu_Rp"].apply(fmt_rp)
        print(display_df.to_string())


══════════════════════════════════════════════════════════════════════
💻 TOP SI CHANNEL ICT — ZONA SUMATERA
══════════════════════════════════════════════════════════════════════
                   Nama_Pemenang Total_Pagu_Rp  Jumlah_Paket  Jumlah_Satuan_Kerja  Jumlah_Instansi
Rank                                                                                              
1          AGRES BATAM TEKNOLOGI     Rp 1.77 M            10                    7                4
2         INTER MEDIALINK SOLUSI     Rp 1.32 M            14                    8                1
3       PT. MULTI DATA PALEMBANG     Rp 1.02 M            12                    6                6
4                        SAYANNA     Rp 1.00 M             5                    5                5
5          AGRES MEDAN TEKNOLOGI   Rp 923.7 Jt            12                    5                5
6            MIKROTEK KOMPUTINDO   Rp 542.1 Jt            16                    9                7
7       AYOOKLIK JBROS TEKNO

In [12]:
# ── Tampilkan semua Top ICT per Bidang ──
for bidang, res in all_bidang_results.items():
    if len(res["top_ict"]) > 0:
        print(f"\n{'═' * 70}")
        print(f"💻 TOP SI CHANNEL ICT — BIDANG {bidang.upper()}")
        print(f"{'═' * 70}")
        display_df = res["top_ict"][["Nama_Pemenang", "Total_Pagu_Rp", 
                                      "Jumlah_Paket", "Jumlah_Satuan_Kerja", 
                                      "Jumlah_Instansi"]].copy()
        display_df["Total_Pagu_Rp"] = display_df["Total_Pagu_Rp"].apply(fmt_rp)
        print(display_df.to_string())


══════════════════════════════════════════════════════════════════════
💻 TOP SI CHANNEL ICT — BIDANG PENDIDIKAN
══════════════════════════════════════════════════════════════════════
                                                    Nama_Pemenang Total_Pagu_Rp  Jumlah_Paket  Jumlah_Satuan_Kerja  Jumlah_Instansi
Rank                                                                                                                               
1                              PT. Telekomunikasi Indonesia, Tbk.   Rp 118.48 M            34                   32                9
2                                          INDONESIA COMNETS PLUS    Rp 63.83 M            11                    8                6
3     PERUSAHAAN PERSEROAN (PERSERO) PT. TELEKOMUNIKASI INDONESIA    Rp 21.66 M            18                   17                6
4                                              SENTRA VIDYA UTAMA     Rp 4.68 M            12                   10                4
5                       